<div style="font-size: 200%; font-weight: bold; color: maroon;">DataFrames en el API de Scala</span></div>
<!-- v 2.1 -- Paulo Villegas, 2021 -->

# Inicialización y carga de datos

Para la inicialización de SparkSQL a partir de Spark 2.0, usamos un `SparkSession`

Cargamos Spark en el kernel y creamos una sesión

In [ ]:
// Load the Spark Jars into the Scala interpreter
def loadSpark(path: Option[String] = None ) = {
    val jardir = path.getOrElse("/opt/spark/current") + "/jars"
    val jars = java.nio.file.Files.list(java.nio.file.Paths.get(jardir)).toArray.map(_.toString);
    val seq = jars.map( fname => ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(fname)) );
    interp.load.cp(seq);
}

loadSpark(sys.env.get("SPARK_HOME"));

In [ ]:
import org.apache.spark.sql._

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org.apache").setLevel(Level.WARN)

In [ ]:
val spark = SparkSession.builder().master("local[*]").getOrCreate()

In [ ]:
spark.getClass

A través del `SparkSession` podemos obtener un `DataFrame` de distintas formas. En este caso leeremos [a partir de CSV](https://spark.apache.org/docs/latest/api/scala/#org.apache.spark.sql.DataFrameReader)

In [ ]:
val df = spark.read.format("csv").
    option("header","true").option("comment","#").option("inferSchema",true).
    load("../DATA/situacion-laboral-titulados-2019.csv")

# Inspección

In [ ]:
df.printSchema()

¿Cuántas filas tenemos?

In [ ]:
df.count

Son pocas filas, podemos verlas:

In [ ]:
df.show( truncate=false )

# Operaciones

In [ ]:
df.groupBy("Rama").sum("Total").show()

A la hora de efectuar operaciones en DataFrames, puede ser necesario usar los operadores definidos sobre el objeto [Column](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Column) del API de DataFrame



Nota: el código contiene dos elementos de sintaxis especial implícita de Spark en Scala:
 * [`$`](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/sql/SQLImplicits$StringToColumn.html) obtiene un objeto `Column` a partir de su nombre 
 * [`===`](https://spark.apache.org/docs/latest/api/scala/org/apache/spark/sql/Column.html#===(other:Any):org.apache.spark.sql.Column) compara dos columnas


In [ ]:
import spark.implicits._

In [ ]:
// Filtro como condición con objeto Scala
val countsByAge = df.filter( $"Tipo" === "privadas").groupBy("Sexo").sum()

// Filtro como condición SQL
//val countsByAge = df.filter( "Tipo = 'privada'").groupBy("sexo").sum()

countsByAge.show()